Installing required libraries

In [ ]:
%pip install langchain langchain_openai pydantic guardrails-ai gradio duckduckgo_search -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.4/234.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9

Importing Secret Value

In [ ]:
from google.colab import userdata
GROQ = userdata.get('GROQ')
GURADRAIL = userdata.get('GURADRAIL')

Setting Up Guardrails Configuration

In [ ]:
!guardrails configure --disable-remote-inferencing --disable-metrics --token {GURADRAIL}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.

            Login successful.

            Get started by installing our RegexMatch validator:
            https://hub.guardrailsai.com/validator/guardrails_ai/regex_match

            You can install it by running:
            guardrails hub install hub://guardrails/regex_match

            Find more validators at https://hub.guardrailsai.com
            


Downloading Specific Validators

In [ ]:
!guardrails hub install hub://guardrails/toxic_language

Installing hub://guardrails/toxic_language...
[  ==] Fetching manifest
[   =] Downloading dependencies
[   =] Running post-install setup[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[  ==] Running post-install setup[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[ ===] Running post-install setupDownloading: "https://github.com/unitaryai/detoxify/releases/download/v0.1.2/unbiased-albert-c8519128.ckpt" to /root/.cache/torch/hub/checkpoints/unbiased-albert-c8519128.ckpt
100% 44.6M/44.6M [00:00<00:00, 118MB/s]
[ ===] Running post-install setup2025-02-21 11:12:16.620901: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
[=   ] Running post-install setupWARNING: All log messages before absl::InitializeLog() is called are written to STDERR
E0000 00:00:1740136336.899595    1462 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to r

Importing the required libraries

In [ ]:
import os
import json
import gradio as gr
from datetime import datetime
from duckduckgo_search import DDGS
from typing import List, Dict, Tuple
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from guardrails.hub import ToxicLanguage
from guardrails import Guard, OnFailAction
from langchain_core.language_models import BaseChatModel

Setting up a env variable for OpenAI API key

In [ ]:
os.environ['OPENAI_API_KEY'] = GROQ

Realtime data access tool with some business logic functions

In [ ]:
class DuckDuckGoSearch:
    def __init__(self):
        self.ddgs = DDGS()

    def _fetch_results(self, query: str, max_results: int = 5) -> List[Dict]:
        try:
            results = self.ddgs.text(keywords=query, max_results=max_results)
            return results
        except Exception as e:
            print(f"Error fetching results: {str(e)}")
            return []

    def get_weather_forecast(self, location: str, date: str) -> List[Dict]:
        query = f"Weather forecast for {location} on {date}"
        return self._fetch_results(query, 10)

    def get_hotel_bookings(
        self, location: str, type: str, checkin: str, checkout: str, budget: str
    ) -> List[Dict]:
        query = f"Hotel bookings in {location} for {type} from {checkin} to {checkout} within {budget}"
        return self._fetch_results(query, 10)

    def get_flight_bookings(
        self,
        origin: str,
        destination: str,
        departure: str,
        return_date: str,
        budget: str,
    ) -> List[Dict]:
        query = f"Flight bookings from {origin} to {destination} on {departure} and return on {return_date} within {budget}"
        return self._fetch_results(query, 10)

Customized tools with the langchain

In [ ]:
# Tools


class DestinationRequest(BaseModel):
    """Request destination recommendations based on user preferences."""

    destination: str = Field(..., description="Selected destination")
    num_people: int = Field(..., description="Number of travelers")
    duration_days: int = Field(..., description="Length of trip in days")
    season: str = Field(..., description="Preferred travel season")
    accommodation_level: str = Field(
        ..., description="Preferred accommodation level (budget/mid-range/luxury)"
    )


class ActivityPlan(BaseModel):
    """Request activity plan based on user preferences."""

    destination: str = Field(..., description="Selected destination")
    num_days: int = Field(..., description="Number of days")
    activities: List[str] = Field(..., description="Preferred activities")
    budget_per_day: float = Field(..., description="Daily budget per person")


class BudgetBreakdown(BaseModel):
    """Request budget breakdown based on user preferences."""

    destination: str = Field(..., description="Selected destination")
    num_people: int = Field(..., description="Number of travelers")
    duration_days: int = Field(..., description="Length of trip in days")
    accommodation_level: str = Field(..., description="Accommodation level")
    activities: List[str] = Field(..., description="Planned activities")
    total_budget: float = Field(..., description="Total budget")


class FlightBooking(BaseModel):
    """Book a flight ticket."""

    origin: str = Field(..., description="Origin airport")
    destination: str = Field(..., description="Destination airport")
    departure: str = Field(..., description="Departure date")
    return_date: str = Field(..., description="Return date")
    budget: str = Field(..., description="Budget for the flight")


class HotelBooking(BaseModel):
    """Book a hotel room."""

    location: str = Field(..., description="Location")
    type: str = Field(..., description="Type of room")
    checkin: str = Field(..., description="Check-in date")
    checkout: str = Field(..., description="Check-out date")
    budget: str = Field(..., description="Budget for the room")


class WeatherForecast(BaseModel):
    """Get weather forecast for a location."""

    location: str = Field(..., description="Location")
    date: str = Field(..., description="Date")


langchain_tools = [
    DestinationRequest,
    ActivityPlan,
    BudgetBreakdown,
    FlightBooking,
    HotelBooking,
    WeatherForecast,
]

Store, Get and Clear Chat History

In [ ]:
class ChatHistory:
    def __init__(self, file_path: str = "chat_history.json"):
        self.file_path = file_path
        self.history = self.load_history()

    def load_history(self) -> List[List[str]]:
        """Load chat history from file if it exists."""
        try:
            with open(self.file_path, "r") as f:
                data = json.load(f)
                return [[msg["user"], msg["bot"]] for msg in data]
        except (FileNotFoundError, json.JSONDecodeError):
            return []

    def save_history(self, history: List[List[str]]):
        """Save chat history to file."""
        data = [
            {"timestamp": datetime.now().isoformat(), "user": msg[0], "bot": msg[1]}
            for msg in history
            if msg[1] is not None  # Only save complete messages
        ]
        with open(self.file_path, "w") as f:
            json.dump(data, f)

    def clear_history(self) -> List[List[str]]:
        """Clear chat history and return empty list."""
        if os.path.exists(self.file_path):
            os.remove(self.file_path)
        self.history = []
        return self.history

    def get_context_string(self) -> str:
        """Convert chat history to a context string for the LLM."""
        context = []
        for user_msg, bot_msg in self.history:
            context.append(f"User: {user_msg}")
            if bot_msg:
                context.append(f"Assistant: {bot_msg}")
        return "\n".join(context)

Prompts

In [ ]:
VALIDATE_PROMPT = """
You are a travel planning assistant. Your task is to analyze the conversation and determine if you have all required information for planning.

Previous Conversation:
{conversation_context}

Latest Query:
"{query}"

Required information:
    - Number of travelers
    - Budget (total or per person)
    - Duration of trip
    - Preferred activities or interests
    - Time of travel (season/month)
    - Accommodation preferences
    - Flight details (optional)

Analyze the conversation history and latest query. Check if all required information is available.

You must respond with a valid JSON object in the following format, and ONLY this format:
{{
"is_complete": false,
"missing_info": ["list", "of", "missing", "details"],
"follow_up_question": "your follow-up question here"
}}

Rules:
1. The response must be a valid JSON object
2. All fields must be present
3. "is_complete" must be boolean
4. "missing_info" must be an array of strings
5. "follow_up_question" must be a string

Consider previous messages when checking for information. If a detail was provided earlier, don't ask for it again.
Make the follow_up_question natural and conversational.
"""

TOOL_PROMPT = """
Create a detailed travel plan based on the following conversation:

Previous Conversation:
{conversation_context}

Latest Query:
{query}

Tool Name and Usage:
    - Destination Request: Recommend destinations based on user preferences
    - Activity Plan: Create an activity plan based on user preferences
    - Budget Breakdown: Provide a budget breakdown based on user preferences
    - Flight Booking: If the user wants to book a flight ticket (optional)
    - Hotel Booking: If the user wants to book a hotel room (optional)
    - Weather Forecast: If the user wants forecast for a location (optional)

"""

FORMAT_PROMPT = """
Based on the following conversation history and tool responses.

Previous Conversation:
{conversation_context}

Latest Query:
{query}

Tool Responses:
{tool_responses}

Additional API Responses:
{additional_api_responses}

IMPORTANT NOTE:
- Do not mention API names or technical details in the response.

If conversation context is available then consider the entire conversation context when creating the plan. Reference and build upon any preferences or details mentioned in earlier messages.
Make the content informative yet engaging while maintaining a professional tone.
"""

Chat core logic

In [ ]:
class Chat:
    def __init__(
        self,
        tools: List[BaseModel],
        chat_model: BaseChatModel,
        ddgs: DuckDuckGoSearch = DuckDuckGoSearch(),
    ):
        self.ddgs = ddgs
        self.chat_history = ChatHistory()
        self.chat_model = chat_model
        self.chat_model_tools = self.chat_model.bind_tools(tools)
        self.guard = Guard().use(
            ToxicLanguage(
                threshold=0.5,
                validation_method="sentence",
                on_fail=OnFailAction.EXCEPTION,
            )
        )

    def _ddgs_search(self, tool_response: str) -> str:
        """Execute search queries based on tool response."""
        try:
            response = ""
            parsed_response = json.loads(tool_response)

            tool_calls = []
            if isinstance(parsed_response, dict):
                if "tool_calls" in parsed_response:
                    tool_calls = parsed_response["tool_calls"]
                elif "additional_kwargs" in parsed_response:
                    if "tool_calls" in parsed_response["additional_kwargs"]:
                        tool_calls = parsed_response["additional_kwargs"]["tool_calls"]

            if not tool_calls:
                return ""

            for tool_call in tool_calls:
                # Extract function details safely
                function_details = tool_call.get("function", {})
                name = function_details.get("name", "")
                arguments = function_details.get("arguments", "{}")

                try:
                    args = (
                        json.loads(arguments)
                        if isinstance(arguments, str)
                        else arguments
                    )
                except json.JSONDecodeError:
                    continue

                if name == "HotelBooking":
                    details = self.ddgs.get_hotel_bookings(
                        args.get("location", ""),
                        args.get("type", ""),
                        args.get("checkin", ""),
                        args.get("checkout", ""),
                        args.get("budget", ""),
                    )
                    if details:
                        response += f"Hotel Details: {json.dumps(details)}\n"

                elif name == "WeatherForecast":
                    details = self.ddgs.get_weather_forecast(
                        args.get("location", ""), args.get("date", "")
                    )
                    if details:
                        response += f"Weather Forecast: {json.dumps(details)}\n"

                elif name == "FlightBooking":
                    details = self.ddgs.get_flight_bookings(
                        args.get("origin", ""),
                        args.get("destination", ""),
                        args.get("departure", ""),
                        args.get("return_date", ""),
                        args.get("budget", ""),
                    )
                    if details:
                        response += f"Flight Details: {json.dumps(details)}\n"

            return response
        except Exception as e:
            print(f"Error fetching DDGS search results: {str(e)}")
            return ""

    def _format_context(self, history: List[List[str]]) -> str:
        """Format conversation history into a context string."""
        context = []
        for user_msg, bot_msg in history:
            context.append(f"User: {user_msg}")
            if bot_msg:
                context.append(f"Assistant: {bot_msg}")
        return "\n".join(context)

    def _validate_query_information(self, query: str, history: List[List[str]]) -> Dict:
        """Validate if all required travel information is provided."""
        history_context = self._format_context(history)

        try:
            validate_prompt = VALIDATE_PROMPT.format(
                conversation_context=history_context, query=query
            )
            response = self.chat_model.invoke(validate_prompt)
            parsed_response = json.loads(response.content)

            required_keys = {"is_complete", "missing_info", "follow_up_question"}
            if not all(key in parsed_response for key in required_keys):
                return {
                    "is_complete": False,
                    "missing_info": ["invalid_response"],
                    "follow_up_question": "Could you please provide more details about your travel plans?",
                }

            return parsed_response
        except json.JSONDecodeError:
            return {
                "is_complete": False,
                "missing_info": ["error"],
                "follow_up_question": "Could you please provide more details about your travel plans?",
            }

    def _execute_tool_response(self, query: str, history: List[List[str]]) -> str:
        """Execute tools based on the validated query."""
        try:
            history_context = self._format_context(history)
            tool_prompt = TOOL_PROMPT.format(
                conversation_context=history_context, query=query
            )

            response = self.chat_model_tools.invoke(tool_prompt)
            return json.dumps(response.additional_kwargs)

        except Exception as e:
            print(f"Error executing tool response: {str(e)}")
            return "I apologize, but I'm having trouble processing your request. Could you please try again?"

    def generate_response(self, query: str, history: List[List[str]]) -> str:
        """Generate a complete travel plan response."""
        try:
            # Guardrails validation for toxic language
            try:
                self.guard.validate(query)
            except Exception:
                return (
                    f"Please refrain from using inappropriate language in your messages like '{query}'. Thank you!",
                    "",
                    "",
                )

            tool_response = self._execute_tool_response(query, history)
            history_context = self._format_context(history)
            additional_api_responses = self._ddgs_search(tool_response)

            formatted_prompt = FORMAT_PROMPT.format(
                conversation_context=history_context,
                query=query,
                tool_responses=tool_response,
                additional_api_responses=additional_api_responses,
            )

            response = self.chat_model.invoke(formatted_prompt)
            return response.content, tool_response, additional_api_responses

        except Exception as e:
            print(f"Error generating response: {str(e)}")
            return (
                "I apologize, but I encountered an error while creating your travel plan. Please try again with your requirements.",
                tool_response if tool_response else "",
                additional_api_responses if additional_api_responses else "",
            )

Interface Setup and addons

In [16]:
def create_interface():
    llm = ChatOpenAI(
        base_url="https://api.groq.com/openai/v1", model="llama-3.2-90b-vision-preview"
    )
    chat_process = Chat(tools=langchain_tools, chat_model=llm)

    def user_message(
        message: str,
        history: List[List[str]],
        tool_responses: List[str],
        api_responses: List[str],
    ) -> Tuple[str, List[List[str]], str, str]:
        history = history or []
        history.append([message, None])
        return "", history, "", ""

    def format_json(json_str: str) -> str:
        """Format JSON string for better readability."""
        try:
            parsed = json.loads(json_str)
            return json.dumps(parsed, indent=2)
        except Exception:
            return json_str

    def bot_message(
        history: List[List[str]], tool_responses: List[str], api_responses: List[str]
    ) -> Tuple[List[List[str]], str, str]:
        if history and history[-1][1] is None:
            previous_history = [h for h in history[:-1] if h[1] is not None]
            response, tool_resp, api_resp = chat_process.generate_response(
                history[-1][0], previous_history
            )
            history[-1][1] = response
            chat_process.chat_history.save_history(history)

            # Format the responses
            formatted_tool_resp = format_json(tool_resp)
            formatted_api_resp = format_json(api_resp)

            return history, formatted_tool_resp, formatted_api_resp
        return history, "", ""

    example_messages = [
        "I'm planning a trip to Europe with 4 friends. We have a budget of $2000 per person and want to visit historical sites and museums. We're flexible with the travel dates.",
        "I'm looking for a beach vacation in the Caribbean for 2 people. Our budget is $3000 for a week-long trip. We enjoy water sports and relaxing on the beach.",
        "Why some people are so dumb to listen",
        "I need a cheaper flight from chennai to madurai",
        "I want to go on a hiking trip in the mountains for 5 days. My budget is $1000 and I prefer to stay in budget accommodations.",
        "I want to go on a solo hiking trip to Mt. Everest for 10 days on June 1st. I have a budget of $5000 and I'm looking to climb the base camp. I would like to stay in mid-range accommodations like lodges. I don't want any activities beside hiking. I'm doing hiking for about 5 years and very fit. I'll bring my own medical equiments. Can you help me plan this trip?",
    ]

    with gr.Blocks(title="Octonomy - Workshop 1", theme=gr.themes.Default(primary_hue=gr.themes.colors.violet, secondary_hue=gr.themes.colors.purple)) as demo:
        # Header with logo and name
        with gr.Row():
                with gr.Column(scale=1):
                    gr.HTML(
                      """
                      <a href="https://www.octonomy.ai/" target="_blank">
                          <img src="https://www.octonomy.ai/wp-content/uploads/2025/01/octonomy-Light-Logo-1.svg"
                              style="width: 150px; cursor: pointer;">
                      </a>
                      """
                  )
        # Control for showing/hiding responses
        with gr.Row():
            show_responses = gr.Checkbox(
                label="Show Response Details", value=False, interactive=True
            )

        # Main container
        with gr.Row():
            # Chat container (dynamic width based on checkbox)
            chat_container = gr.Column(scale=1, visible=True)
            with chat_container:
                chatbot = gr.Chatbot(
                    value=chat_process.chat_history.load_history(),
                    height=480,
                    show_label=False,
                )

                with gr.Row():
                    msg = gr.Textbox(
                        placeholder="Enter your travel requirements...",
                        show_label=False,
                        scale=6,
                        container=False,
                    )
                    submit = gr.Button("Send", scale=1, variant="primary")
                    clear = gr.Button("Clear Chat", scale=1)  

                with gr.Row():
                    
                    gr.Examples(
                        examples=example_messages,
                        inputs=msg,
                        label="Example Queries",
                        examples_per_page=2,
                    )

            # Response panels (visibility toggled by checkbox)
            responses_container = gr.Column(scale=1, visible=False)
            with responses_container:
                with gr.Tabs():
                    with gr.Tab("Tool Details"):
                        tool_responses = gr.Code(
                            label="Processing Details",
                            language="json",
                            show_label=False,
                            lines=15,
                        )
                    with gr.Tab("API Results"):
                        api_responses = gr.Code(
                            label="External Data",
                            language="json",
                            show_label=False,
                            lines=15,
                        )

        # Toggle response panels visibility
        def toggle_responses(show: bool):
            return {
                responses_container: gr.update(visible=show),
                chat_container: gr.update(scale=2 if show else 1),
            }

        show_responses.change(
            toggle_responses,
            show_responses,
            [responses_container, chat_container],
        )

        # Event handlers
        msg.submit(
            user_message,
            [msg, chatbot, tool_responses, api_responses],
            [msg, chatbot, tool_responses, api_responses],
            queue=False,
        ).then(
            bot_message,
            [chatbot, tool_responses, api_responses],
            [chatbot, tool_responses, api_responses],
        )

        submit.click(
            user_message,
            [msg, chatbot, tool_responses, api_responses],
            [msg, chatbot, tool_responses, api_responses],
            queue=False,
        ).then(
            bot_message,
            [chatbot, tool_responses, api_responses],
            [chatbot, tool_responses, api_responses],
        )

        def clear_all():
            chat_process.chat_history.clear_history()
            return [], "", ""

        clear.click(
            clear_all,
            None,
            [chatbot, tool_responses, api_responses],
            queue=False,
        )

    return demo

Launch the Chat window

In [ ]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:
if __name__ == "__main__":
    demo = create_interface()
    demo.queue()
    demo.launch(inline=False, share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6122c2f037a64ffe9f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
